In [5]:
import os
import torch
import hydra
import lightning.pytorch as pl
from lightning.pytorch.callbacks import LearningRateMonitor
import importlib


from omegaconf import OmegaConf,open_dict
import omegaconf



config_home_dir = './config'

cfg_ae_global_config = omegaconf.OmegaConf.load(config_home_dir+'/ae/global_config.yaml')
cfg_ae_data = omegaconf.OmegaConf.load(config_home_dir+'/ae/data.yaml')
cfg_ae_vq_vae = omegaconf.OmegaConf.load(config_home_dir+'/ae/vq_vae.yaml')
cfg_ae_global_config = omegaconf.OmegaConf.load(config_home_dir+'/ae/global_config.yaml')
cfg_ae_model = omegaconf.OmegaConf.load(config_home_dir+'/ae/model.yaml')


cfg_denoiser_data = omegaconf.OmegaConf.load(config_home_dir+'/denoiser/data.yaml')
cfg_denoiser_encode = omegaconf.OmegaConf.load(config_home_dir+'/denoiser/encoder.yaml')
cfg_denoiser_global_config = omegaconf.OmegaConf.load(config_home_dir+'/denoiser/global_config.yaml')
cfg_denoiser_model = omegaconf.OmegaConf.load(config_home_dir+'/denoiser/model.yaml')



cfg = OmegaConf.merge(
    cfg_ae_data,
    cfg_ae_vq_vae,
    cfg_ae_global_config,
    cfg_ae_model,
    cfg_denoiser_data,
    cfg_denoiser_encode,
    cfg_denoiser_global_config,
    cfg_denoiser_model
)

data_home_dir = '/disk2/data/breaking_bad/'
data_home_dir = '/disk2/data/shape_dataset/'

data_type_name = 'shape'

cfg.ckpt_path= None
cfg.project_root_path = data_home_dir
cfg.experiment_name = 'shape_epoch10'
cfg.experiment_output_path: '${project_root_path}/output/autoencoder/${experiment_name}'



cfg.data.data_dir = data_home_dir+f'data/pc_data/{data_type_name}/train/'
cfg.data.data_val_dir = data_home_dir+f'data/pc_data/{data_type_name}/val/'
cfg.data.mesh_data_dir = data_home_dir+'data/'
cfg.data.data_fn = data_type_name+".{}.txt"

cfg.data.batch_size = 3
cfg.data.val_batch_size= 3
cfg.trainer.devices=1
cfg.trainer.max_epochs =  3
cfg.trainer.check_val_every_n_epoch =  1



def init_callbacks(cfg):
    checkpoint_monitor = hydra.utils.instantiate(cfg.checkpoint_monitor)
    lr_monitor = LearningRateMonitor(logging_interval="epoch")
    return [checkpoint_monitor, lr_monitor]

    
pl.seed_everything(cfg.train_seed, workers=True)
# create directories for training outputs
os.makedirs(os.path.join(cfg.experiment_output_path, "training"), exist_ok=True)


Seed set to 123


In [12]:
import numpy as np

import importlib
import puzzlefusion_plusplus.vqvae.dataset.pc_dataset
importlib.reload(puzzlefusion_plusplus.vqvae.dataset.pc_dataset)

from puzzlefusion_plusplus.vqvae.data.data_module import DataModule
from puzzlefusion_plusplus.vqvae.dataset import dataset
from puzzlefusion_plusplus.vqvae.dataset import pc_dataset
from puzzlefusion_plusplus.vqvae.dataset.dataset import build_geometry_dataloader


# initialize data
data_module = DataModule(cfg)




100%|██████████| 34/34 [00:00<00:00, 464.77it/s]


In [13]:
_item = None
train_count = 0
val_count = 0
for nth_batch, batch in enumerate(data_module.train_dataloader()):
    #print(nth_batch, len(batch), batch.keys(), batch['mesh_file_path'])
    for _i in range(len(batch['mesh_file_path'])):
        print(batch["data_id"][_i], batch["mesh_file_path"][_i])
        _item_gt = batch["part_pcs_gt"][_i].tolist()
        _item_rt = batch["part_pcs"][_i].tolist()
        #print(_item)
        filename_gt = f'{data_home_dir}/data/test_data/{batch["data_id"][_i]}_gt.npz'
        filename_rt = f'{data_home_dir}/data/test_data/{batch["data_id"][_i]}_rt.npz'
        print(filename_gt)
        print(filename_rt)
        np.savez(filename_gt, _item_gt) 
        np.savez(filename_rt, _item_rt) 
 

tensor(1) shape/cube/0/cube_32_16_0_1_1
/disk2/data/shape_dataset//data/test_data/1_gt.npz
/disk2/data/shape_dataset//data/test_data/1_rt.npz
tensor(25) shape/cube/0/cube_512_256_0_1_3
/disk2/data/shape_dataset//data/test_data/25_gt.npz
/disk2/data/shape_dataset//data/test_data/25_rt.npz
tensor(35) shape/cube/0/cube_512_256_1_1_1
/disk2/data/shape_dataset//data/test_data/35_gt.npz
/disk2/data/shape_dataset//data/test_data/35_rt.npz
tensor(67) shape/cube/0/cube_512_256_3_3_3
/disk2/data/shape_dataset//data/test_data/67_gt.npz
/disk2/data/shape_dataset//data/test_data/67_rt.npz
tensor(93) shape/ellipsoid/0/ellipsoid_512_128_64_64_2_1_3
/disk2/data/shape_dataset//data/test_data/93_gt.npz
/disk2/data/shape_dataset//data/test_data/93_rt.npz
tensor(26) shape/cube/0/cube_512_256_0_2_1
/disk2/data/shape_dataset//data/test_data/26_gt.npz
/disk2/data/shape_dataset//data/test_data/26_rt.npz
tensor(82) shape/ellipsoid/0/ellipsoid_512_128_64_64_1_1_3
/disk2/data/shape_dataset//data/test_data/82_gt.

In [ ]:
cfg.model.encoder_weights_path = cfg.experiment_output_path+'/training/last.ckpt'
cfg.ckpt_path= None
cfg.experiment_output_path = data_home_dir+'output/denoiser/${experiment_name}/'

cfg.trainer.strategy='ddp'

print(OmegaConf.to_yaml(cfg))



In [ ]:
from puzzlefusion_plusplus.denoiser.dataset.dataset import build_geometry_dataloader as denoiser_dataloader

In [ ]:
pl.seed_everything(cfg.train_seed, workers=True)

# create directories for training outputs
os.makedirs(os.path.join(cfg.experiment_output_path, "training"), exist_ok=True)

# initialize data
train_loader, val_loader = denoiser_dataloader(cfg)



In [ ]:
for nth_batch, batch in enumerate(train_loader):
    print(nth_batch, batch['mesh_file_path'], batch['data_id'])

In [4]:

# initialize model
model = hydra.utils.instantiate(cfg.model.model_name, cfg)

if cfg.model.encoder_weights_path is not None:
    encoder_weights = torch.load(cfg.model.encoder_weights_path)['state_dict']
    model.encoder.load_state_dict({k.replace('ae.', ''): v for k, v in encoder_weights.items()})
    # freeze the encoder
    for param in model.encoder.parameters():
        param.requires_grad = False

# initialize logger
logger = hydra.utils.instantiate(cfg.logger)

# initialize callbacks
callbacks = init_callbacks(cfg)

# initialize trainer
trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    **cfg.trainer
)

# check the checkpoint
if cfg.ckpt_path is not None:
    assert os.path.exists(cfg.ckpt_path), "Error: Checkpoint path does not exist."

# start training
trainer.fit(
    model=model, 
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    ckpt_path=cfg.ckpt_path
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


NameError: name 'train_loader' is not defined